In [ ]:
#ETH using Finta signals with SVM and DTC

In [99]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests
from finta import TA

In [100]:
# Load environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=1095)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PKHBLCB7QI73L4912M0G", "Aq2JSZktyKcSKaFWq1VxIPn3EOFJB0ICvB7fdmpE", ALPACA_API_BASE_URL, api_version="v2")

In [101]:
# Set the ticker symbol and the number of shares to buy
ticker = "ETHUSD"
number_of_coins = 1

# Make API call
ETH_prices = api.get_crypto_bars(ticker, '1day',
                         start=_3YearsAgo.isoformat(),
                         end=None,
                         limit=None).df

In [102]:
#Filter data by exchange
ETH_prices = ETH_prices[ETH_prices.exchange == 'CBSE']

In [103]:
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [104]:
# Create a signals_df DataFrame that is a copy of the ixn_df Dataframe
signals_df = ETH_prices.copy()

In [105]:
signals_df.drop(columns=['exchange','volume','trade_count','vwap'],inplace=True)

In [136]:
signals_df.tail()

,open,high,low,close,fast,slow,Signal,Entry/Exit,Actual Returns
timestamp,,,,,,,,,
2022-10-23 05:00:00+00:00,1309.95,1370.90,1299.47,1347.90,1301.784969,1281.672409,1.0,0.0,0.028986
2022-10-24 05:00:00+00:00,1347.89,1359.86,1324.04,1346.60,1311.016790,1282.760913,1.0,0.0,-0.000964
2022-10-25 05:00:00+00:00,1346.60,1525.38,1339.39,1480.84,1350.103392,1294.186079,1.0,0.0,0.099688
2022-10-26 05:00:00+00:00,1480.83,1595.44,1477.47,1561.92,1401.074955,1311.254298,1.0,0.0,0.054753
2022-10-27 05:00:00+00:00,1562.00,1580.00,1533.77,1556.64,1441.375762,1326.989576,1.0,0.0,-0.003380


In [107]:
# Set the short window and long windows
fast_window = 15
slow_window = 50

# Add the technical indicators (Double Exponential Moving Average) for the fast and slow windows
signals_df["fast"] = TA.DEMA(signals_df, fast_window)
signals_df["slow"] = TA.DEMA(signals_df, slow_window)

In [108]:
signals_df[["fast","slow"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [109]:
signals_df.head()

,open,high,low,close,fast,slow
timestamp,,,,,,
2019-10-29 05:00:00+00:00,191.03,192.20,186.35,187.62,187.620000,187.620000
2019-10-30 05:00:00+00:00,187.63,187.63,180.05,183.38,184.303378,184.398024
2019-10-31 05:00:00+00:00,182.38,183.09,180.46,181.17,182.131239,182.307486
2019-11-01 05:00:00+00:00,181.17,184.99,177.39,183.19,182.328266,182.389362
2019-11-02 05:00:00+00:00,183.26,185.00,183.25,183.84,182.762668,182.716075


In [110]:
# Set the Signal column
signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the fast window is greater than (or crosses over) the slow Window
# where 0 is when the fast window is under the slow window
signals_df["Signal"][fast_window:] = np.where(
    signals_df["fast"][fast_window:] > signals_df["slow"][fast_window:], 1.0, 0.0)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.iloc[120:180, :]

,open,high,low,close,fast,slow,Signal,Entry/Exit
timestamp,,,,,,,,
2020-02-26 06:00:00+00:00,223.78,223.85,209.26,221.30,261.402969,253.677120,1.0,0.0
2020-02-27 06:00:00+00:00,221.24,239.00,220.44,227.63,254.645137,253.104813,1.0,0.0
2020-02-28 06:00:00+00:00,227.32,231.70,213.63,227.37,248.988123,252.517104,0.0,-1.0
2020-02-29 06:00:00+00:00,217.36,225.78,215.99,222.55,243.189501,251.562650,0.0,0.0
2020-03-01 06:00:00+00:00,217.34,222.00,215.75,221.23,238.118063,250.534930,0.0,0.0
2020-03-02 06:00:00+00:00,232.12,232.47,227.85,230.11,236.052619,250.228517,0.0,0.0
2020-03-03 06:00:00+00:00,230.09,232.92,220.00,223.75,232.887784,249.422517,0.0,0.0
2020-03-04 06:00:00+00:00,224.49,229.48,224.49,229.18,231.594586,249.059081,0.0,0.0
2020-03-05 06:00:00+00:00,229.17,234.47,226.69,228.64,230.440191,248.651148,0.0,0.0


In [111]:
signals_df["Signal"].value_counts()

1.0    561
0.0    534
Name: Signal, dtype: int64

In [112]:
signals_df["Entry/Exit"].value_counts()

 0.0    1063
 1.0      16
-1.0      15
Name: Entry/Exit, dtype: int64

In [113]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["fast", "slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot


:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [114]:
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [115]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['fast', 'slow']].shift().dropna()

# Review the DataFrame
X.head()

,fast,slow
timestamp,,
2019-10-30 05:00:00+00:00,187.620000,187.620000
2019-10-31 05:00:00+00:00,184.303378,184.398024
2019-11-01 05:00:00+00:00,182.131239,182.307486
2019-11-02 05:00:00+00:00,182.328266,182.389362
2019-11-03 05:00:00+00:00,182.762668,182.716075


In [116]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

1.0    561
0.0    534
Name: Signal, dtype: int64

In [117]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-10-30 05:00:00+00:00


In [118]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end)

2021-10-30 05:00:00+00:00


In [119]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,fast,slow
timestamp,,
2019-10-30 05:00:00+00:00,187.620000,187.620000
2019-10-31 05:00:00+00:00,184.303378,184.398024
2019-11-01 05:00:00+00:00,182.131239,182.307486
2019-11-02 05:00:00+00:00,182.328266,182.389362
2019-11-03 05:00:00+00:00,182.762668,182.716075


,fast,slow
timestamp,,
2021-10-26 05:00:00+00:00,4174.800523,3909.430972
2021-10-27 05:00:00+00:00,4226.249210,3951.005378
2021-10-28 05:00:00+00:00,4200.319623,3968.186618
2021-10-29 05:00:00+00:00,4261.466195,4011.707883
2021-10-30 05:00:00+00:00,4305.394724,4050.980380


In [120]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,fast,slow
timestamp,,
2021-10-31 05:00:00+00:00,4347.306337,4090.225634
2021-11-01 05:00:00+00:00,4346.946066,4116.119241
2021-11-02 05:00:00+00:00,4376.677795,4150.639192
2021-11-03 05:00:00+00:00,4440.544153,4196.374420
2021-11-04 05:00:00+00:00,4500.197863,4242.131630


,fast,slow
timestamp,,
2022-10-23 05:00:00+00:00,1290.270243,1280.495278
2022-10-24 05:00:00+00:00,1301.784969,1281.672409
2022-10-25 05:00:00+00:00,1311.016790,1282.760913
2022-10-26 05:00:00+00:00,1350.103392,1294.186079
2022-10-27 05:00:00+00:00,1401.074955,1311.254298


In [121]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [122]:
from sklearn import svm
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [123]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       0.99      0.60      0.75       208
         1.0       0.65      0.99      0.78       154

    accuracy                           0.77       362
   macro avg       0.82      0.80      0.77       362
weighted avg       0.85      0.77      0.76       362



In [124]:
# Use the pct_change function to generate returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Create an empty predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['SVM Returns'] = signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,SVM Returns
timestamp,,,
2021-10-31 05:00:00+00:00,1.0,-0.032032,-0.032032
2021-11-01 05:00:00+00:00,1.0,0.032097,0.032097
2021-11-02 05:00:00+00:00,1.0,0.040322,0.040322
2021-11-03 05:00:00+00:00,1.0,0.008608,0.008608
2021-11-04 05:00:00+00:00,1.0,-0.027580,-0.027580


,Predicted,Actual Returns,SVM Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.028986,0.028986
2022-10-24 05:00:00+00:00,1.0,-0.000964,-0.000964
2022-10-25 05:00:00+00:00,1.0,0.099688,0.099688
2022-10-26 05:00:00+00:00,1.0,0.054753,0.054753
2022-10-27 05:00:00+00:00,1.0,-0.003380,-0.003380


In [133]:
# Plot the actual returns versus the SVM strategy returns 
(1 + predictions_df[["Actual Returns", "SVM Returns"]]).cumprod().hvplot(title="ETH returns with DEMA/SVM",height=400,width=900)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [126]:
# Import a new classifier from SKLearn
from sklearn import tree

# Initiate the model instance
dtc_model=tree.DecisionTreeClassifier()

In [127]:
# Fit the model using the training data
dtc_model = dtc_model.fit(X_train_scaled,y_train)

# Use the testing dataset to generate the predictions for the new model
dtc_pred = dtc_model.predict(X_test_scaled)
# Review the model's predicted values
dtc_pred[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [128]:
# Use a classification report to evaluate the model using the predictions and testing data
dtc_testing_report = classification_report(y_test, dtc_pred)

# Print the classification report
print(dtc_testing_report)

              precision    recall  f1-score   support

         0.0       0.78      0.42      0.55       208
         1.0       0.52      0.84      0.64       154

    accuracy                           0.60       362
   macro avg       0.65      0.63      0.59       362
weighted avg       0.67      0.60      0.59       362



In [129]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
final_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
final_df["Predicted"]=dtc_pred

# Add the actual returns to the DataFrame
final_df["Actual Returns"]=signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
final_df["DTC Returns"]=final_df["Actual Returns"]*final_df["Predicted"]

# Review the DataFrame
display(final_df.head())
display(final_df.tail())

,Predicted,Actual Returns,DTC Returns
timestamp,,,
2021-10-31 05:00:00+00:00,1.0,-0.032032,-0.032032
2021-11-01 05:00:00+00:00,1.0,0.032097,0.032097
2021-11-02 05:00:00+00:00,1.0,0.040322,0.040322
2021-11-03 05:00:00+00:00,1.0,0.008608,0.008608
2021-11-04 05:00:00+00:00,1.0,-0.027580,-0.027580


,Predicted,Actual Returns,DTC Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.028986,0.028986
2022-10-24 05:00:00+00:00,1.0,-0.000964,-0.000964
2022-10-25 05:00:00+00:00,1.0,0.099688,0.099688
2022-10-26 05:00:00+00:00,1.0,0.054753,0.054753
2022-10-27 05:00:00+00:00,1.0,-0.003380,-0.003380


In [134]:
# Plot the actual returns versus the strategy returns
(1 + final_df[["Actual Returns", "DTC Returns"]]).cumprod().hvplot(title="ETH returns with DEMA/DTC",height=400,width=900)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)